In [0]:
%pip install -U -r requirements.txt
%restart_python

## Create a Python-based tool

In [0]:
from unitycatalog.ai.core.databricks import DatabricksFunctionClient


def translate_sku(old_sku: str) -> str:
    """
    Translates a pre-2024 SKU formatted as "OLD-XXX-YYYY" to the new SKU format "NEW-YYYY-XXX".

    Args:
        old_sku (str): The old SKU in the format "OLD-XXX-YYYY".

    Returns:
        str: The new SKU in the format "NEW-YYYY-XXX", or an error message if the format is incorrect.
    """
    import re
    # Define the regex pattern for the old SKU format
    pattern = r"^OLD-([A-Z]{3})-(\d{4})$"
    # Match the old SKU against the pattern
    match = re.match(pattern, old_sku)
    if not match:
        return "Error: Invalid SKU format"
    # Extract the letter code and numeric part
    letter_code, numeric_part = match.groups()
    # Construct the new SKU by swapping the parts
    new_sku = f"NEW-{numeric_part}-{letter_code}"
    return new_sku


client = DatabricksFunctionClient()

result = client.create_python_function(
    func=translate_sku,
    catalog="agents_demo",
    schema="playground",
    replace=True
)

print(result)

## Create a SQL-based tool

In [0]:
%sql
CREATE
OR REPLACE FUNCTION agents_demo.playground.user_orders (
  user_id STRING
  COMMENT 'User ID to find orders for'
) RETURNS TABLE(
  amount INT,
  order_id STRING,
  item_count INT,
  creation_date TIMESTAMP
)
COMMENT 'Finds a user\'s recent orders: the amount, order ID, item count, and creation date based on the provided user ID.' RETURN
SELECT
  amount,
  order_id,
  item_count,
  creation_date
FROM
  agents_demo.data.customer_orders
WHERE
  user_id = user_id;

## Other tools

In [0]:
def python_exec(code: str) -> str:
  """
  Executes Python code in the sandboxed environment and returns its stdout. The runtime is stateless and you can not read output of the previous tool executions. i.e. No such variables "rows", "observation" defined. Calling another tool inside a Python code is NOT allowed. Use only standard python libraries and these python libraries: bleach, chardet, charset-normalizer, defusedxml, googleapis-common-protos, grpcio, grpcio-status, jmespath, joblib, numpy, packaging, pandas, patsy, protobuf, pyarrow, pyparsing, python-dateutil, pytz, scikit-learn, scipy, setuptools, six, threadpoolctl, webencodings, user-agents, cryptography.

  Args:
    code (str): Python code to execute. Remember to print the final result to stdout.

  Returns:
    str: The output of the executed code.
  """
  import sys
  from io import StringIO
  sys_stdout = sys.stdout
  redirected_output = StringIO()
  sys.stdout = redirected_output
  exec(code)
  sys.stdout = sys_stdout
  return redirected_output.getvalue()


client = DatabricksFunctionClient()

result = client.create_python_function(
    func=python_exec,
    catalog="agents_demo",
    schema="playground",
    replace=True
)

print(result)

In [0]:
from typing import Dict
def send_customer_apology(user_name: str, message: str) -> Dict[str, str]:
  """
  Sends an apology note to the customer with the appropiate valued gift certificate based on their current situation.

  Args:
    user_name (str): user name to send the apology to
    message (str): any additional notes to include the standard apology note

  Returns:
    str: The output of the executed code.
  """
  return {
    'status': 'success',
    'refund_amount': '100',
    'message_sent': f'We are so sorry, here is $100.  Please try us again. {message}',
  }

client = DatabricksFunctionClient()

result = client.create_python_function(
    func=send_customer_apology,
    catalog="agents_demo",
    schema="playground",
    replace=True
)

print(result)

In [0]:
%sql
CREATE
OR REPLACE FUNCTION agents_demo.playground.search_product_documentation (
  topic STRING
  COMMENT 'Topic to find product documentation about, can be a product name or a query about a product'
) RETURNS TABLE(
  indexed_doc string,
  product_category string,
  product_doc string,
  product_id string,
  product_name string,
  product_sub_category string,
  search_score DOUBLE
)
COMMENT 'Finds product documentation' RETURN
SELECT
  *
FROM
  VECTOR_SEARCH(
    index => "agents_demo.data.product_docs_index",
    query => topic,
    num_results => 5
  )

In [0]:
%sql
CREATE
OR REPLACE FUNCTION agents_demo.playground.search_customer_support_transcripts (
  topic STRING
  COMMENT 'Topic to find recent call center transcripts about'
) RETURNS TABLE(
  issue_area string,
  issue_category string,
  issue_sub_category string,
  product_category string,
  product_sub_category string,
  conversation string,
  id string,
  timestamp timestamp,
  user_id string,
  search_score double
)
COMMENT 'Finds a recent call center transcripts about a topic' RETURN
SELECT
  *
except(concatenated_string)
FROM
  VECTOR_SEARCH(
    index => "agents_demo.data.customer_support_transcripts_index",
    query => topic,
    num_results => 5
  )